# 8-9. 프로젝트 - Movielens 영화 추천 실습

# 1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import numpy as np
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)
print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)


In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')

) 분석해 봅시다.
ratings에 있는 유니크한 영화 개수
rating에 있는 유니크한 사용자 수
가장 인기 있는 영화 30개(인기순)

In [6]:
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# 총 3883개의 영화가 등록되어 있습니다. 많진 않군요.
movies['title'].nunique()

3883

In [8]:
#총 유저수는 6040명입니다
ratings['user_id'].nunique()

6039

In [9]:
# 드라마와 코메디 장르가 많이 등록되어 있네요
movies_count = movies.groupby('genre')['title'].count()
movies_count.sort_values(ascending=False).head(10)

genre
Drama             843
Comedy            521
Horror            178
Comedy|Drama      162
Comedy|Romance    142
Drama|Romance     134
Documentary       116
Thriller          101
Action             65
Drama|Thriller     63
Name: title, dtype: int64

In [10]:
genre_musical = movies[movies['genre'].str.contains("Musical")]
genre_musical.head(40)
#저는 뮤지컬 장르를 좋아하니까 genre칼럼에서 Musical 영화들만 찾아볼게요.

,movie_id,title,genre
47,48,Pocahontas (1995),Animation|Children's|Musical|Romance
72,73,"Mis�rables, Les (1995)",Drama|Musical
105,107,Muppet Treasure Island (1996),Adventure|Children's|Comedy|Musical
197,199,"Umbrellas of Cherbourg, The (Parapluies de Che...",Drama|Musical
239,242,Farinelli: il castrato (1994),Drama|Musical
323,327,Tank Girl (1995),Action|Comedy|Musical|Sci-Fi
342,346,Backbeat (1993),Drama|Musical
360,364,"Lion King, The (1994)",Animation|Children's|Musical
547,551,"Nightmare Before Christmas, The (1993)",Children's|Comedy|Musical
584,588,Aladdin (1992),Animation|Children's|Comedy|Musical


In [11]:
# 인기 많은 영화들의 id를 뽑아봤습니다.
hot_rated = ratings.groupby('movie_id')['count'].count()
hot_rated.sort_values(ascending=False).head(10)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
Name: count, dtype: int64

In [12]:
#조금 더 보기 편하게 만들기 위해 movie_id를 기준으로 데이터프레임을 합쳐봅니다. 
check = pd.merge(ratings,movies,on ='movie_id', how ='left')
check

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986),Drama|War
836474,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,1096,4,956715648,Sophie's Choice (1982),Drama


In [ ]:
# 아까 뽑은 id들의 제목을 확인해봅니다.
hot_rated = check.groupby('title')['movie_id'].count()
hot_rated.sort_values(ascending=False).head(10)

# 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
별점을 시청횟수로 해석해서 생각하겠습니다.
또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [ ]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Terminator 2: Judgment Day (1991)' , 'Toy Story (1995)' ,'Nightmare Before Christmas, The (1993)' ,'Sound of Music, The (1965)' ,'Mary Poppins (1964)']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['MIN']*5, 'title': my_favorite, 'count':[5]*5})

if not  check.isin({'user_id':['MIN']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = check.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

# 4) CSR matrix를 직접 만들어 봅시다.

In [ ]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [ ]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['MIN'])    
print(artist_to_idx['Terminator 2: Judgment Day (1991)'])

In [ ]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

temp_movies_data = data['title'].map(artist_to_idx.get).dropna()
if len(temp_movies_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movies_data
else:
    print('artist column indexing Fail!!')

data

In [ ]:
#학습시에 사용하지 않을 데이터를 지워줍니다.
del data["movie_id"]
del data["timestamp"]
#movie_id와 'timestamp'를 지워줍니다. genre는 활용할 예정이기에 남겨둡니다.

In [ ]:
#genre의 결측치 부분을  Musical로 채워줍니다.
data = data.fillna('Musical')

In [ ]:
#확인해봅니다. 잘됐네요. 결과물에 뮤지컬 영화들 추천이 많이 나올까요?
data.tail(10)

In [ ]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movies = data['title'].nunique()
csr_data = csr_matrix((data['count'],(data.user_id, data.title)),shape= (num_user, num_movies))
csr_data

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [ ]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

In [ ]:
# 모델 훈련
als_model.fit(csr_data_transpose)

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.    
이 과정을 통해서는 모델 학습 과정을 유추해볼 수 있었습니다.

In [ ]:
Pocahontas, WE = artist_to_idx['Pocahontas (1995)'], artist_to_idx['Waiting to Exhale (1995)']
Pocahontas_vector, WE_vector = als_model.item_factors[Pocahontas], als_model.item_factors[WE]

In [ ]:
Pocahontas_vector

In [ ]:
WE_vector

In [ ]:
#내적하는 코드
np.dot(Pocahontas_vector, WE_vector)
#포카혼타스는 애니메이션 뮤지컬이고 사랑을 기다리며는 로맨스 드라마 장르입니다. 일부러 서로 먼 걸 골라봤어요. 
#예상대로 낮은 점수가 나오네요.

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [ ]:
favorite_artist = 'Sound of Music, The (1965)'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=10)
similar_artist

In [ ]:
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

In [ ]:
get_similar_artist('Sound of Music, The (1965)')

추천값이 뮤지컬 영화들인지 궁금해서 구글링을 해봤습니다.     
Rosemary's Baby (1968)를 제외하면 ***장르가 뮤지컬이네요.***

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.    
이번에는 제가 고른 특정 영화가 아니라 유저인 ***MIN***이 좋아할 것 같은 영화들을 추천받아봅니다.      
뮤지컬 영화들 사이에 이질적인 영화를 하나 섞어봤어요. 터미네이터 혼자 SF 액션 영화입니다. 과연 잘 찾아낼까요?
```
'Terminator 2: Judgment Day (1991)' ,    
'Toy Story (1995)' ,   
'Nightmare Before Christmas, The (1993)' ,    
'Sound of Music, The (1965)' ,    
'Mary Poppins (1964)'
```


In [ ]:
user = user_to_idx['MIN']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

In [ ]:
[idx_to_artist[i[0]] for i in movie_recommended]

In [ ]:
Matrix = artist_to_idx['Matrix, The (1999)']
explain = als_model.explain(user, csr_data, itemid=Matrix)

In [ ]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]
#'Matrix'를 추천하는데 가장 크게 기여를 한 건 'Terminator 2: Judgment Day'입니다.   
#둘 다 미래를 배경으로 하는 공상과학 액션 영화네요!
#단 하나의 영화만 있어도 추천에는 문제가 없군요!

# summary  
Movielens를 기반으로 한 영화 추천 시스템을 만들어봤습니다.     
과정은 다음과 같습니다.
1. 데이터 전처리      
학습을 위해 컬럼내의 정보들을 정수 index화 해줍니다.
2. rating과 movie dataframe을 합쳐준다.     
별점(평가기준)을 기준으로 합쳐줍니다. 
3. csr matrix를 형성해줍니다.
4. 모델 학습    
implicit 안의 als(AlternatingLeastSquares) 모델을 활용합니다.    
als 모델은 input으로 ***item X user*** 꼴의 matrix를 받습니다.
5. AlternatingLeastSquares 클래스 속 ```similar_items``` 사용해보기 
6. AlternatingLeastSquares 클래스 속 ```recommend``` 사용해보기

     


```Matrix Factorization(MF, 행렬분해)``` 모델에 대해 알게되었습니다.   
벡터를 곱했을 때 1에 가까워야 모델이 잘 학습했다는 것을 의미합니다.                   
                        
          
               
```CSR(Compressed Sparse Row) Matrix```
유저 수 X 아티스트 수만큼의 정보 안에는 유저가 들어보지 않은      
아티스트에 대한 정보까지 모두 행렬에 포함되어 계산되기에     
평가행렬은 ***Sparse Matrix***(평가행렬 내의 대부분의 공간이 0인 행렬)이 됩니다.    
***CSR Matrix***는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여    
메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다. 

# 후기      

행렬을 내적해서 추천 시스템을 만든다는 발상이 신기했다. 처음에는 user_id와 counts(ratings) 그리고 movie_title을 제외한   
정보들을 제거하려 했었으나 스트리밍 사이트들에 **장르기반** 추천 시스템이 있다는 것을 기억해내고 남겨두기로 했다. 나와 비슷한 취향을 가진 
누군가의 데이터에 기대기보다는 내 자료를 (겨우 5개지만) 일관성있게 musical 장르로 모아 학습시키니 만족스러운 결과를 얻을 수 있었다.